<a href="https://colab.research.google.com/github/SeriKim12/Beer-Reccomendation-Chatbot/blob/main/%EB%B2%84%ED%8A%B8%EB%AA%A8%EB%8D%B8%ED%95%99%EC%8A%B50610.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pwd

/content


In [ ]:
## downgrade tensorflow version
%tensorflow_version 1.x

In [ ]:
import os, re
import argparse

% cd /content/drive/MyDrive

# get tokenizer python script
from tokenizationK import FullTokenizer

## get tokenizer vocab file
tokenizer = FullTokenizer(vocab_file="bert-module/assets/vocab.korean.rawtext.list")

/content/drive/MyDrive


In [ ]:
slot_pattern = re.compile(r"/([^/]+);([^/]+)/")
multi_spaces = re.compile("\s{2,}")

def process_file(file_path, output_dir):
    """
    단방향 데이터가 있는 file_path을 argument로 주면 가공을 한 이후에
    output_dir 아래에 2개의 파일(seq.in, seq.out)을 저장해 주는 함수.
    """
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)

    data = open(file_path).read().splitlines()

    # line별로 process를 해준 뒤,
    processed_data = [process_line(line, tokenizer) for line in data]
    #(토큰화된 문장, 슬롯 0 0 슬롯 0 0 )
    ## ----------------- 문제 2 ---------------- ##
    # tokens = 알맞은 코드를 써주세요
    # tags = 알맞은 코드를 써주세요
    tokens = list(map(lambda x: x[0], processed_data))
    tags = list(map(lambda x: x[1], processed_data))
    ## ---------------------------------------- ##


    # seq_in : 토큰들로만 이루어진 파일
    # seq_out : 태그들로만 이루어진 파일
    seq_in = os.path.join(output_dir, "seq.in")
    seq_out = os.path.join(output_dir, "seq.out")

    with open(seq_in, "w") as f:
        f.write("\n".join(tokens)+ "\n")

    with open(seq_out, "w") as f:
        f.write("\n".join(tags)+ "\n")

In [ ]:
# for, if문을 중심으로 이해해보자!!
def process_line(sentence, tokenizer):
    """
    데이터를 라인별로 처리해 주는 함수이다.
    라인을 주게 되면, (토큰, 슬롯)

    예를 들어 "/인물;한지민/과 /인물;한예슬/ 나오는 드라마 있어?" 같은 input을 받게 되면,
        ('한 지민 과 한예 슬 나오 는 드라마 있 어 ?', '인물 인물 O 인물 인물 O O O O O O')와 같은 (토큰, 태그)쌍으로 된 결과값을 반환한다.
    """
    # 설명 --> /ㅇㅇ;ㅇㅇ/를 전부 찾음 --> slot_pattern = re.compile()
    #['/인물;한지민/', '/인물;한예슬/']
    slot_pattern_found = slot_pattern.findall(sentence)
    # 설명: /ㅇㅇ;ㅇㅇ/ --> /슬롯/  --> 슬롯인지 아닌지만 판단
    # '/슬롯/ /슬롯/ /슬롯/ 나오는 드라마 있어'
    line_refined = slot_pattern.sub("/슬롯/", sentence)
    tokens = ""
    tags = ""
    slot_index = 0
    
    # 설명: for문에 넣은 문장: /슬롯/과 /슬롯/ 나오는 드라마 있어? --> split
    # word --> 1. /슬롯/과       2.  /슬롯/       3. 나오는    4. 드라마.....
    # 슬롯의 3가지 경우: 1. /슬롯/   2. /슬롯/과 
    for word in line_refined.split():

        # 기본적으로 전처리를 잘 거쳤다면, "/슬롯/과"처럼 뒤에 조사가 붙는 경우는 있어도, "안녕/슬롯/" 이렇게 앞에 붙지는 않을 것.
        # 즉 /슬롯/~을 잡는 if문
        if word.startswith("/"):

            # r"/(.+?);(.+?)/" 이렇게 그룹화했기 때문
            slot, entity = slot_pattern_found[slot_index]
            slot_index += 1

            # 엔티티를 토크나이즈 한 후, 토큰별로 태그를 추가해 준다.
            entity_tokens = " ".join(tokenizer.tokenize(entity))

            # 설명: tokens
            tokens += entity_tokens + " "
            tags += (slot + " ") * len(entity_tokens.split())

            # /슬롯/ /슬롯/의 /슬롯/에서    --> 이런 것들 중에 /슬롯/의 /슬롯/에서
            # 즉 /슬롯/ 뒤에 조사가 붙은 경우
            # 조사에 대해서 추가적으로 토큰 및 태그를 추가해 준다.
            # /슬롯/ --> 이 경우는 if word.endswith("/"): 에 해당한다.
            if not word.endswith("/"):
                # 우선 "/" 뒤에 오는 조사를 찾아 주고
                josa = word[word.rfind("/")+1:]
                josa_tokens = " ".join(tokenizer.tokenize(josa))

                tokens += josa_tokens + " "
                tags += "O " * len(josa_tokens.split())

        # 안녕/슬롯/ --> 전처리가 잘 되지 않은 경우
        # /슬롯/
        elif "/" in word:

            prefix = word.split("/")[0]
            tokenized_prefix = " ".join(tokenizer.tokenize(prefix))
            tokens += tokenized_prefix + " "
            tags += "O " * len(tokenized_prefix.split())

            slot, entity = slot_pattern_found[slot_index]
            slot_index += 1

            entity_tokens = " ".join(tokenizer.tokenize(entity))

            tokens += entity_tokens + " "
            tags += (slot + " ") * len(entity_tokens.split())
            
        # 나머지: 즉, 슬롯과 관계 없는 일반적인 단어: 
        # "/슬롯/"도 아니고, "안녕/슬롯"도 아님 --> 그냥 일반적인 토큰들 ("드라마", "있어?")
        else:
            word_tokens = " ".join(tokenizer.tokenize(word))

            tokens += word_tokens + " "
            tags += "O " * len(word_tokens.split())

    tokens = multi_spaces.sub(" ", tokens.strip())
    tags = multi_spaces.sub(" ", tags.strip())

    # 만일 토큰의 개수와 태그의 개수가 맞지 않다면 본래 라인과 더불어 토큰/태그들을 프린트해준다.
    ## ----------------- 문제 3 ---------------- ##
    # [if 문으로 시작하는 코드 작성 ! ]  
    if len(tokens.split()) != len(tags.split()):
        print(sentence)
        print("\t" + tokens + "\t", len(tokens.split()))
        print("\t" + tags + "\t", len(tags.split()))

    return tokens, tags

In [ ]:
% cd /content/drive/MyDrive
process_file('data.txt', 'seq')

/content/drive/MyDrive


In [ ]:
import argparse

# 인자값을 받을 수 있는 인스턴스 생성
# 인자값: 터미널에서 파이썬 실행할 때 쓰는 일종의 input   ex) 경로
parser = argparse.ArgumentParser(description='사용법 테스트입니다.')

# 입력받을 인자값 등록
# --target: 이름   /   required=True/False : 필수 여부 
parser.add_argument('--target', '-t', required=True, help='어느 것을 요구하냐')
parser.add_argument('--env', '-e',  required=False, default='dev', help='실행환경은 뭐냐')
# python test.py --target "안녕하세요" --env "반가워요"
# python test.py -t "안녕하세요"

# 입력받은 인자값을 args에 저장 (type: namespace)
args = parser.parse_args()

# 입력받은 인자값 출력 (앞서 썼던 이름 --target, --env --> args.target / args.env에 따라 사용)
print(args.target)
print(args.env)

usage: ipykernel_launcher.py [-h] --target TARGET [--env ENV]
ipykernel_launcher.py: error: the following arguments are required: --target/-t


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# $ python3 argparse_test.py
# usage: argparse_test.py [-h] --target TARGET [--env ENV]
# argparse_test.py: error: the following arguments are required: --target

# $ python3 argparse_test.py -h
# usage: argparse_test.py [-h] --target TARGET [--env ENV]

# 사용법 테스트입니다.

# optional arguments:
#   -h, --help       show this help message and exit
#   --target TARGET  어느 것을 요구하냐
#   --env ENV        실행환경은 뭐냐

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
! pwd

/content


In [ ]:
# github에 codes 폴더를 올려주세요
# etri checkpoint를 팀원과 공유해주세요.

# git clone "클론할 링크" "저장할 링크"
! git clone https://github.com/SeriKim12/Beer-Reccomendation-Chatbot.git /content/drive/MyDrive/bert

Cloning into '/content/drive/MyDrive/bert'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 43 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [ ]:
! ls

삭제하기  drive  sample_data


In [ ]:
% cd /content/drive/MyDrive/
! ls

In [ ]:
! pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.8 MB 4.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lucid 0.3.10 requires umap-learn, which is not installed.
tensorflow 1.15.2 requires gast==0.2.2, but you have gast 0.5.3 which is incompatible.
lucid 0.3.10 requires numpy<=1.19, but you have numpy 1.19.5 which is incompatible.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumenta

pre-trained KorBERT 모델을 모듈로 export

In [ ]:
% cd /content/drive/MyDrive
! python codes/codes/export_korbert/bert_to_module.py -i codes/codes/004_bert_eojeol_tensorflow -o bert-module

/content/drive/MyDrive
From codes/codes/export_korbert/bert_to_module.py:30: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

From /content/drive/MyDrive/codes/codes/export_korbert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

0.02 ---------
From /content/drive/MyDrive/codes/codes/export_korbert/modeling.py:194: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

From /content/drive/MyDrive/codes/codes/export_korbert/modeling.py:435: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

From /tensorflow-1.15.2/python3.7/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
From /content/drive

seq을 train test val 로 나누기 - in out 파일이 안나눠짐

In [ ]:
! pip install split-folders
import splitfolders

% cd /content/drive/MyDrive
splitfolders.ratio('/content/drive/MyDrive/seq', output = 'splited_seq', ratio=(0.8, 0.1, 0.1))

train_test_split으로 나누기 - 안됨...

In [ ]:
from sklearn.model_selection import train_test_split

% cd /content/drive/MyDrive/seq

train_in, test_in, train_out, test_out = train_test_split('seq.in', 'seq.out', test_size=0.2)
test_in, val_in, test_out, val_out = train_test_split(test_in, test_out, test_size=0.5)

/content/drive/MyDrive/seq


ValueError: ignored

스벅 주문 챗봇 팀 train test split 코드

In [ ]:
import pandas as pd
import os
import numpy as np
import random

% cd /content/drive/MyDrive

def train_test_val_split():

    data_dir = '/content/drive/MyDrive'

    in_li = []
    in_ap = in_li.append

    out_li = []
    out_ap = out_li.append

    train_in = []
    train_inap = train_in.append
    train_out = []
    train_outap = train_out.append

    test_in = []
    test_inap = test_in.append
    test_out = []
    test_outap = test_out.append

    val_in = []
    val_inap = val_in.append
    val_out = []
    val_outap = val_out.append



    with open(os.path.join(data_dir,'seq/seq.in'),'r') as f:
        for sentence in f.readlines():
            in_ap(sentence.strip())

    with open(os.path.join(data_dir,'seq/seq.out'),'r') as f:
        for sentence in f.readlines():
            out_ap(sentence.strip())

    num_li = list(range(len(in_li)))
    random.shuffle(num_li)

    val = int(len(num_li) * 0.1)
    test = int(len(num_li) * 0.1)
    train = len(num_li) - (val + test)

    val_idx = num_li[:val]
    test_idx = num_li[val:val+test]
    train_idx = num_li[val+test:]

    for i in val_idx:
        val_inap(in_li[i])
        val_outap(out_li[i])

    for i in test_idx:
        test_inap(in_li[i])
        test_outap(out_li[i])

    for i in train_idx:
        train_inap(in_li[i])
        train_outap(out_li[i])

    with open(os.path.join(data_dir,'splited_seq/test/seq.in'),'w') as f:
        for line in test_in:
            f.write(line+'\n')

    with open(os.path.join(data_dir,'splited_seq/test/seq.out'),'w') as f:
        for line in test_out:
            f.write(line+'\n')

    with open(os.path.join(data_dir,'splited_seq/val/seq.in'),'w') as f:
        for line in val_in:
            f.write(line+'\n')

    with open(os.path.join(data_dir,'splited_seq/val/seq.out'),'w') as f:
        for line in val_out:
            f.write(line+'\n')

    with open(os.path.join(data_dir,'splited_seq/train/seq.in'),'w') as f:
        for line in train_in:
            f.write(line+'\n')

    with open(os.path.join(data_dir,'splited_seq/train/seq.out'),'w') as f:
        for line in test_out:
            f.write(line+'\n')

if __name__ == "__main__":
    train_test_val_split()

/content/drive/MyDrive


In [ ]:
len('/content/drive/MyDrive/seq/seq.out')

34

In [ ]:
len('/content/drive/MyDrive/splited_seq/test/seq.out')

47

In [ ]:
len('/content/drive/MyDrive/splited_seq/val/seq.out')

46

In [ ]:
len('/content/drive/MyDrive/splited_seq/train/seq.out')

48

todo 문제 풀이

In [ ]:
y_slots = np.array([[[0.00003171, 0.00454775, 0.9950228, 0.00019214, 0.00020573],
                     [0.00000268, 0.9998349, 0.00004407, 0.00001392, 0.00010439],
                     [0.00000056, 0.9999397, 0.00002303, 0.00000644, 0.00003024],
                     [0.00000094, 0.99993575, 0.00002988, 0.00001607, 0.00001725]]]).reshape((4,-1))
max(y_slots[1])

0.9998349

In [ ]:
import numpy as np
import sklearn

y_slots = np.array([[[0.00003171, 0.00454775, 0.9950228, 0.00019214, 0.00020573],
                     [0.00000268, 0.9998349, 0.00004407, 0.00001392, 0.00010439],
                     [0.00000056, 0.9999397, 0.00002303, 0.00000644, 0.00003024],
                     [0.00000094, 0.99993575, 0.00002988, 0.00001607, 0.00001725]]]).reshape((4,-1, 1))
#slots_score = np.array([[0.9950228, 0.9998349, 0.9999397, 0.99993575]])
slots = np.array([['아티스트', 'O', 'O', 'O']]).reshape((4,1))
print(slots[0])
print(y_slots.shape[0])

#print(x) for lambda x : y_slots@x == slots_score

slots_score = [max(y_slots[i])*len(slots[i]) for i in range(y_slots.shape[0])]
slots_score
#a = [len(slots[i]) for i in range(y_slots.shape[0])]
#print(a)

['아티스트']
4


[array([0.9950228]),
 array([0.9998349]),
 array([0.9999397]),
 array([0.99993575])]

In [ ]:
import numpy as np
import sklearn

y_slots = np.array([[[0.00003171, 0.00454775, 0.9950228, 0.00019214, 0.00020573],
                     [0.00000268, 0.9998349, 0.00004407, 0.00001392, 0.00010439],
                     [0.00000056, 0.9999397, 0.00002303, 0.00000644, 0.00003024],
                     [0.00000094, 0.99993575, 0.00002988, 0.00001607, 0.00001725]]])

y_slots.shape

(1, 4, 5)

In [ ]:
import numpy as np
import sklearn

y_slots = np.array([[[0.00003171, 0.00454775, 0.9950228, 0.00019214, 0.00020573],
                     [0.00000268, 0.9998349, 0.00004407, 0.00001392, 0.00010439],
                     [0.00000056, 0.9999397, 0.00002303, 0.00000644, 0.00003024],
                     [0.00000094, 0.99993575, 0.00002988, 0.00001607, 0.00001725]]])

print(y_slots)
print(y_slots[0][0])

slots = np.array([['아티스트', 'O', 'O', 'O']])
print(slots[0])
print(y_slots.shape[0])

slots_score = [[max(y_slots[i][j]) for j in range(y_slots.shape[1])]*len(slots[i]) for i in range(y_slots.shape[0])]
print(slots_score)

slots_score2 = [[max(y_slots[i][j]) for j in range(y_slots.shape[1])] for i in range(y_slots.shape[0])]
print(slots_score2)

[[[3.1710000e-05 4.5477500e-03 9.9502280e-01 1.9214000e-04 2.0573000e-04]
  [2.6800000e-06 9.9983490e-01 4.4070000e-05 1.3920000e-05 1.0439000e-04]
  [5.6000000e-07 9.9993970e-01 2.3030000e-05 6.4400000e-06 3.0240000e-05]
  [9.4000000e-07 9.9993575e-01 2.9880000e-05 1.6070000e-05 1.7250000e-05]]]
[3.171000e-05 4.547750e-03 9.950228e-01 1.921400e-04 2.057300e-04]
['아티스트' 'O' 'O' 'O']
1
[[0.9950228, 0.9998349, 0.9999397, 0.99993575, 0.9950228, 0.9998349, 0.9999397, 0.99993575, 0.9950228, 0.9998349, 0.9999397, 0.99993575, 0.9950228, 0.9998349, 0.9999397, 0.99993575]]
[[0.9950228, 0.9998349, 0.9999397, 0.99993575]]


In [ ]:
slots_score = [[max(y_slots[i][j])]*len(slots[i]) for i in range(y_slots.shape[0]) for j in range(y_slots.shape[1])]
print(slots_score)

[[0.9950228, 0.9950228, 0.9950228, 0.9950228], [0.9998349, 0.9998349, 0.9998349, 0.9998349], [0.9999397, 0.9999397, 0.9999397, 0.9999397], [0.99993575, 0.99993575, 0.99993575, 0.99993575]]


In [ ]:
y_slots = np.array([[[0.00003171, 0.00454775, 0.9950228, 0.00019214, 0.00020573],
                     [0.00000268, 0.9998349, 0.00004407, 0.00001392, 0.00010439],
                     [0.00000056, 0.9999397, 0.00002303, 0.00000644, 0.00003024],
                     [0.00000094, 0.99993575, 0.00002988, 0.00001607, 0.00001725]]])

slots = np.array([['아티스트', 'O', 'O', 'O']])
print(len(slots[0]))

slots_score = [max(y_slots[i][j])*len(slots[i]) for i in range(y_slots.shape[0]) for j in range(y_slots.shape[1])]
print(slots_score)

4
[3.9800912, 3.9993396, 3.9997588, 3.999743]


Fine-tuing 훈련

In [ ]:
% cd /content/drive/MyDrive
! python codes/codes/train_bert_finetuning.py -t seq -v splited_seq/val -s finetuned -e 1

/content/drive/MyDrive

2022-06-12 10:33:14.924361: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-06-12 10:33:14.924664: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1abb100 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-12 10:33:14.924708: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-12 10:33:14.927454: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-12 10:33:14.936429: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-12 10:33:14.936502: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (98abad4440f9): /proc/driver/nvidia/version does not exist
read data ...
train_text_arr[0:2] : ['

모델 평가

In [ ]:
! pip install 'h5py==2.10.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 4.4 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
% cd /content/drive/MyDrive
! python codes/codes/eval_bert_finetuned.py -m finetuned -d splited_seq/test

/content/drive/MyDrive


2022-06-12 12:53:23.226652: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-06-12 12:53:23.229021: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1beb2c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-12 12:53:23.229073: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-12 12:53:23.232299: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-12 12:53:23.245059: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-12 12:53:23.245131: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (98abad4440f9): /proc/driver/nvidia/version does not exist
initializing!
is_bert : True

initial

In [ ]:
import os
a='맛은 /taste;안 신/거 도수는 /abv;8도/인 거 중에서 /type;흑맥주/에 /flavor;스모키한/향 맥주 있어?'
a.strip().splitlines()

['맛은 /taste;안 신/거 도수는 /abv;8도/인 거 중에서 /type;흑맥주/에 /flavor;스모키한/향 맥주 있어?']

Inference 훈련 모델에 문장 넣어보기

In [ ]:
% cd /content/drive/MyDrive
! python codes/codes/inference.py -m finetuned

/content/drive/MyDrive

2022-06-12 16:31:14.166203: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-06-12 16:31:14.166417: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3297480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-12 16:31:14.166454: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-12 16:31:14.168645: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-12 16:31:14.176763: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-12 16:31:14.176807: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (98abad4440f9): /proc/driver/nvidia/version does not exist
initializing!
is_bert : True

initiali